In [ ]:
# SPL - Public to public – international trusted group  

! pip install tenseal cryptography pandas

import random
import pandas as pd
import tenseal as ts
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

# Function to generate synthetic Suspicious Person List (SPL) data
def generate_synthetic_spl(size=100):
  person_ids = [random.randint(10000000, 99999999) for _ in range(size)]
  suspicious_scores = [round(random.uniform(0.0, 20.0), 2) for _ in range(size)]

  return pd.DataFrame({
      'person_id': person_ids,
      'suspicious_score': suspicious_scores
    })

# Function to initialize the Homomorphic Encryption context
def initialize_he_context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, 7)
    context.global_scale = 2 ** 40
    context.generate_galois_keys()
    return context

# Function to encrypt SPL data using Homomorphic Encryption
def encrypt_spl_data(context, spl_data):
    encrypted_spl = []
    for score in spl_data['suspicious_score']:
        encrypted_score = ts.ckks_vector(context, [score])
        encrypted_spl.append(encrypted_score)
    return encrypted_spl

# Function to compare two encrypted SPLs using Homomorphic Encryption
def compare_spl_data(encrypted_spl1, encrypted_spl2):
    results_summary = []
    matched_ids = []
    matched_scores = []

    for enc_score1, enc_score2 in zip(encrypted_spl1, encrypted_spl2):
        comparison_result = enc_score1 - enc_score2
        decrypted_result = comparison_result.decrypt()[0]

        # If the decrypted result is close to 0, it indicates a match
        if abs(decrypted_result) < 1e-7:
            results_summary.append(True)
            # Store the matched IDs and scores for reporting
            matched_ids.append((spl1['person_id'].iloc[len(results_summary)-1], spl2['person_id'].iloc[len(results_summary)-1]))
            matched_scores.append((spl1['suspicious_score'].iloc[len(results_summary)-1], spl2['suspicious_score'].iloc[len(results_summary)-1]))
        else:
            results_summary.append(False)

    return results_summary, matched_ids, matched_scores

# Function to encrypt data using symmetric encryption (AES)
def symmetric_encrypt(data, key):
    backend = default_backend()
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data.encode()) + padder.finalize()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return iv + encrypted_data

# Function to decrypt data using symmetric encryption (AES)
def symmetric_decrypt(encrypted_data, key):
    backend = default_backend()
    iv = encrypted_data[:16]
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data[16:]) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()
    return data.decode()

# Main function to demonstrate the full process
def main():
    spl1 = generate_synthetic_spl(5000)
    spl2 = generate_synthetic_spl(5000)

    # Print the synthetic SPL data for inspection
    # print("SPL 1 Data:")
    # print(spl1)
    # print("\nSPL 2 Data:")
    # print(spl2)

    # Initialize the homomorphic encryption context
    context = initialize_he_context()

    # Encrypt the SPL data using homomorphic encryption
    encrypted_spl1 = encrypt_spl_data(context, spl1)
    encrypted_spl2 = encrypt_spl_data(context, spl2)

    # Compare the encrypted SPLs to find matches
    comparison_results, matched_ids, matched_scores = compare_spl_data(encrypted_spl1, encrypted_spl2)

    # Print matched IDs and scores
    print("\nMatched IDs and Scores:")
    for id_pair, score_pair in zip(matched_ids, matched_scores):
        id1, id2 = id_pair
        score1, score2 = score_pair
        print(f"ID: {id1}, Score in SPL1: {score1}, Score in SPL2: {score2}")

    # Create a summary of the results (e.g., number of matches)
    summary = f"Number of matches: {sum(comparison_results)}"

    # Encrypt the summary using symmetric encryption (AES)
    symmetric_key = os.urandom(32)
    encrypted_summary = symmetric_encrypt(summary, symmetric_key)

    # Decrypt the summary (for demonstration purposes)
    decrypted_summary = symmetric_decrypt(encrypted_summary, symmetric_key)

    # Output the decrypted summary
    print("\nDecrypted Summary:", decrypted_summary)

# Execute the main function
if __name__ == "__main__":
    main()
